# Expectation Maxmiztion




In [1]:
# importing the libraries we will use:

import numpy as np
import random
import math
import pandas as pd

# two additionals for the Lestrade code:
import re
import string

**1. write a simulator as a positive control**

Write a script that simulates N=1000000 observed read counts, following the model specified above, for any Arc locus structure (i.e. lengths Li) and any transcript abundances τ. (You can assume that there are 10 segments and isoforms, though my version of this script will allow that to vary too.)

Use you script to generate a test data set for known model parameters τ,L that you've chosen.



Lets read the provided data table with pandas and store it for future use:

In [2]:
# Using pandas to store our data into a datafile, 
df_data = pd.read_table('w09-data.out.txt', # File name
                            delim_whitespace=True, # Whether it is whitespace delimited
                            skiprows=1, # Number of rows to skip
                            header = None, # Row that has the header (None since we are going to provide it ourselves)
                            names = ['arc','nothing','length','segments'], # Column names
                            index_col=0 # Set the row names using the first column: gene name in this case
                           )

I have no idea whether this was an intended step for the problem set, or whether I am doing it unecessarily long, but here I will do some tricks to clean out the data and get it ready to use in our generative model:

In [3]:
# Number of observed read counts to simulate:
## Including it here because we need it in order to divide the transcript abundances into nucleotide abundances
N = 1000000

# Cleaning up the data with a few tricks:
df_data = df_data.reset_index()  # we do this to avoid using the gene names as indexes and be able to store in list 
data = df_data.values.tolist()

del data[10]

for i in range(10):
    del data[i][1]
  
for i in range(10):
    del data[10+i][2]
    
for i in range(10):
    del data[10+i][2]
    
arcs = data[0:10]
nus = data[10:20]

# Getting lengths into one list:
lengths = []
for i in arcs:
    int_i = int(i[1])
    lengths.append(int_i)

# Getting our nus ready
for i in nus:
    i[1] = float(i[1])/N

n_1 = []  # empty list for nu letters
n_2 = []  # empty list for nu values
for i in nus: 
    n_1.append(i[0])
    n_2.append(i[1])   
 
nu = [n_1, n_2]

a_1 = []  # empty list for arcs[0]
a_2 = []  # empty list for arcs[1]
a_3 = []  # empty list for arcs[2]

for i in arcs:
    a_1.append(i[0])
    a_2.append(i[1]) 
    a_3.append(i[2]) 
    
lens = [a_1, a_2, a_3]

# Storing number of genes here:
number_genes = len(lens[0])


Let's begin by writing a function that generates N read counts. This will give us our test data set for known model parameters τ,L. This is our **generative model**.

In [4]:
## Change from for loop to np.random.choice(N) to make it more efficient

def simulate_reads(N, theta):
    '''
    Given N (the number of reads to sample),
    and theta, which includes nu (nucleotide abundance) and L (transcript lengths),
    Simulate reads
    '''    
    # Unpacking our two parameters from theta to use them in our function: 
    nus = theta[0]                   # nucleotide abundances     
    L = theta[1]                     # list storing lengths for each transctipt

    # Create empty list of reads:
    reads = []
    
    # writing it as a for loop:
    for i in range(N):
        # 1. Choose a transcript G=i with probability V_i:
        transcript = np.random.choice(nus[0], 1, p = nus[1])  # choose a transcript based on the nucleotide abundances
        index = nus[0].index(transcript)                      # index for the transcript (0-9)
        index_seg = arcs[index][2]                            # find the segments that compose that transcript
        list_of_letters = list(index_seg)                     # separate segments into list of letters
        
        # 2. Choose a segment from that transcript with equal probability:
        segment = np.random.choice(list_of_letters, 1)        # pick uniformly from the list of segments in the transcript
        step = list(segment)                                  # extra step to convert from np.array to list
        read = step[0]                                        # appending and also removing "double brackets"
        # 3. Generate a read (counting that segment and mapping a read to it)
        ## simplified model means the segment maps to our read directly: 
        reads.append(read)                                 

    # Return list of size N with our reads:
    return reads
    
    

Now that we have defined our function let's run it with our desired parameters:

In [5]:
# Number of observed read counts to simulate:
N = 1000000

# Storing both parameters in variable "theta" (previously computed above)
theta = [nu, lens]

# Running our function and storing the output
sim_reads = simulate_reads(N, theta)

**2. calculate the log likelihood**

Write a function that calculates the log likelihood: the log probability of the observed data (the observed read counts rk) if the model and its parameters were known (i.e. τ,L), for a given locus structure of Arc.

Calculate and show the log likelihood of one of your generated test data sets, for your known parameter values.

Use Lestrade's approach (and his code, if you like) to estimate abundances of each Arc isoform in your test data set. Compare to your true τ. (Terrible, right?) Calculate and show the log likelihood given what Lestrade would estimate for the τ, compared to the log likelihood for the true τ in your positive control.

You should observe that the true τ parameter values give a much better log likelihood than the Lestrade et al. estimates, because the Lestrade et al. estimates are poor.

Writing our function to calculate the log likelihood for a given locus structure of Arc:

In [6]:
## keep adding and then just log at the end
## we want one P for every read, and then log each one and add all of them

def log_likelihood(R, theta):
    '''
    Given R (reads) and 
    theta (nucleotide abundances and transcript lengths),
    Calculate the total log likelihood of the model
    '''
    # Unpacking our parameters from theta to use them in our function with ease: 
    abundances = theta[0][1]         # our 10 nucleotide abundances
    arcs = theta[1][2]               # segments of the 10 arcs (i.e. 'ABCD')
    lengths = theta[1][1]            # length of the 10 arcs

    ll = 0
    for read in R:
        joint = 0  # joint initial
        # For each read, compute a log likelihood for each of the 10 arcs:
        j = 0 # counter variable that will range 0-9 to keep track of which arc we are dealing with
        for arc in arcs:
            if read in arc:
                pre_joint = (1/int(lengths[j]))*abundances[j]
                joint = joint + pre_joint
            j=j+1
        ll = ll + np.log(joint)
    
    return -ll

Now let's specify our parameters for our log likelihood function and run it with our previously simulated reads:

In [7]:
# As a reminder, theta is: 
theta = [nu, lens]

# Using our function to compute our list of ll's
ll = log_likelihood(sim_reads, theta)

# Getting our total log likelihood:
print('Total log likelihood summed over the ten arcs: ')
print(ll)

Total log likelihood summed over the ten arcs: 
2140469.7194456058


Using Lestrade's code to estimate abundances for each Arc isoform in the test data set we generated above.

In [8]:
# Parse the input file.
with open('w09-data.out.txt') as f:
    #   The first line is "The <n> transcripts of the sand mouse Arc locus"
    line  = f.readline()
    match = re.search(r'^The (\d+) transcripts', line)
    T     = int(match.group(1))

    # The next T lines are 
    #   <Arcn>  <true_tau> <L> <structure>
    # tau's may be present, or obscured ("xxxxx")
    tau       = np.zeros(T)
    L         = np.zeros(T).astype(int)
    tau_known = True   # until we see otherwise
    for i in range(T):
        fields    = f.readline().split()
        if fields[1] == "xxxxx":
            tau_known = False
        else:
            tau[i] = float(fields[1])
        L[i]      = int(fields[2])

    # after a blank line,
    # 'The <n> read sequences':
    line  = f.readline()
    line  = f.readline()
    match = re.search(r'The (\d+) read sequences', line)
    N     = int(match.group(1))

    # the next T lines are 
    #  <read a-j> <count>
    r = np.zeros(T).astype(int)
    for k in range(T):
        fields = f.readline().split()
        r[k]   = fields[1]


S = T    # S = R = T : there are T transcripts (Arc1..Arc10), S segments (A..J), R reads (a..j)
R = T
Slabel   = list(string.ascii_uppercase)[:S]               # ['A'..'J']        : the upper case labels for Arc locus segments 
Tlabel   = [ "Arc{}".format(d) for d in range(1,T+1) ]    # ['Arc1'..'Arc10'] : the labels for Arc transcript isoforms
Rlabel   = list(string.ascii_lowercase)[:T]               # ['a'..'j']        : lower case labels for reads


# Count how often each segment A..J is used in the isoforms i
# We'll use that to split observed read counts across the isoforms
# that they might have come from.
#
segusage = np.zeros(S).astype(int)
for i in range(T):
    for j in range(i,i+L[i]): 
        segusage[j%S] += 1


# Naive analysis:
#
c  = np.zeros(T)
for i in range(T):
    for k in range(i,i+L[i]):
        c[i] += (1.0 / float(segusage[k%S])) * float(r[k%S])  # For each read k, assume read k-> segment j,
                                                              # and assign 1/usage count to each transcript
                                                              # that contains segment j.
Z       = np.sum(c)
est_nu  = np.divide(c, Z)       # nucleotide abundance

print(c)

est_tau = np.divide(est_nu, L)  # convert to TPM, transcript abundance
est_tau = np.divide(est_tau, np.sum(est_tau))

# Print a table of the resulting estimates for tau
for i in range(T):
    print ("{0:10s}  {1:5.3f}".format(Tlabel[i], est_tau[i]))
tau = nu[1]

[115419.66666667  52349.33333333  81691.66666667 130411.83333333
 150381.16666667 131258.16666667 102686.5         83889.5
  79741.16666667  72171.        ]
Arc1        0.085
Arc2        0.077
Arc3        0.080
Arc4        0.096
Arc5        0.111
Arc6        0.129
Arc7        0.151
Arc8        0.123
Arc9        0.078
Arc10       0.071


Comparing this above results to our true abundances tau, we realize Lestrade's are terrible:

In [9]:
# Our abundances:
for i in range(T):
    print ("{0:10s}  {1:5.3f}".format(Tlabel[i], nu[1][i]))

Arc1        0.089
Arc2        0.069
Arc3        0.088
Arc4        0.100
Arc5        0.057
Arc6        0.086
Arc7        0.198
Arc8        0.213
Arc9        0.061
Arc10       0.039


We'll do some formatting here to get the Lestrade data in the correct format:

In [10]:
lestrade_nus = [nu[0], est_tau]

Using our log likelihood function defined above to calculate Lestrade's total log likelihood using his estimated taus:

In [11]:
# The theta parameters for Lestrade: 
theta = [lestrade_nus, lens]

# The reads are the ones from our generated test data set:
R = sim_reads

# Using our function to compute Lestrade's list of ll's
lestrade_ll = log_likelihood(R, theta)

# Getting Lestrade's total log likelihood:
print('Total log likelihood for Lestrade: ')
print(lestrade_ll)

Total log likelihood for Lestrade: 
2162952.708061929


Compared to our model, which uses the true τ parameter values, we can see that Lestrade's total log likelihood is a  larger number, which means it is much less likely than ours (remember we are in log space!). 

In [12]:
# Showing difference between our log likelihood and Lestrade's (remember we are in log space)
ll_diff = lestrade_ll - ll
    
# Displaying the differences:
print(ll_diff)

22482.9886163231


**3. estimate isoform abundances by EM**
Write a script that estimates unknown isoform abundances τi for each isoform Arc1..Arc10, given read counts rk and the structure of the Arc locus including the lengths Li, using expectation maximization.

Apply your script to the data in the Lestrade et al. supplementary data file. Show your estimated τi.

What do you think of Lestrade et al.'s conclusion that all 10 mRNA transcripts are expressed at about the same level? What are the most abundant two transcripts, and how much of the population to they account for? What are the least abundant two transcripts?

- **Step 1**: initialize $\theta$ to anything (can be random)


- **Step 2**: (expectation) calculate $c_i = \sum_n P(G_n=i|R_n,\theta)$


- **Step 3**: (maximization) calculate updated $\theta_i = c_i/N$


- **Step 4**: repeat EM until converged

First step is to set our initial parameters theta:

In [13]:
# Initializing initial parameters theta 'randomly'. 
# This is our initial guess at the probabilty of each read to belong to each transcript

theta[0][1] = [1/number_genes] * number_genes

# reminder of what the variable theta holds:
theta

[[['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'],
  [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]],
 [['Arc1',
   'Arc2',
   'Arc3',
   'Arc4',
   'Arc5',
   'Arc6',
   'Arc7',
   'Arc8',
   'Arc9',
   'Arc10'],
  ['4', '2', '3', '4', '4', '3', '2', '2', '3', '3'],
  ['ABCD', 'BC', 'CDE', 'DEFG', 'EFGH', 'FGH', 'GH', 'HI', 'IJA', 'JAB']]]

In [14]:
## Strategy for EM:

# Idea:
   # We want to run our EM until our LL converges and is optimized


# Now do expectation step:

  # How many reads do we expect mapped to each transcript? c_i 

  # then update by: counts assigned to transcript A/total number of counts

  # after update calculate ll again and check for convergence (ends until it converges)
    
    # each read has some probabilty of belonging to each cluster
    # first calculate the ten probabilities or counts for each read. and then sum over all N reads
    # final result should be an array of 10 abundances

  # expectation step is in picture; then you update parameters by normalizing the counts for each transcript
    

Our second step is to write our function for expectation, which should return a list of counts

In [15]:
def expectation(R, theta):
    '''
    Given R (reads) and 
    theta (nucleotide abundances and transcript lengths),
    Run EM until convergence
    '''
    # Unpacking our parameters from theta to use them in our function with ease: 
    abundances = theta[0][1]         # our 10 nucleotide abundances
    arcs = theta[1][2]               # segments of the 10 arcs (i.e. 'ABCD')
    lengths = theta[1][1]            # length of the 10 arcs
    N = len(R)                       # number of reads

    ## this is all groundwork for the marginalization:
    
    # Summing over all genes to get probability:    
    list_of_strings = []     # empty list to store all the transcripts in (i.e. 'ABC' or 'IAJ')  
    for i in arcs:    # loop through each arc
        list_of_strings.append(list(i))    # add all of them to one list and separate into chars for easy counting

    list_of_strings = np.sum(list_of_strings) # this makes it such that they are all one long list   

    # Making a list with each count to loop through. It will look like: [A,B,C,D,E,F,G,H,I,J]
    unique_segments = list(dict.fromkeys(list_of_strings))  # remove duplicates to create this list (and avoid hardcoding)

    # Empty list to store counts in:
    a_counts = []

    # loop through our arcs and count ocurrences of each segment:
    for i in unique_segments: # for each segment [A,B,C,D,E,F,G,H,I,J] count ocurrences in the ten arcs
        a_counts.append(list_of_strings.count(i))

    # we also want to store the total number of segments:
    total_num_segs = np.sum(a_counts)

    seg_probs = []  # empty list for probabilities of each of the ten segments 
    for i in a_counts:    # probability for each segment (ocurrences of segment/total number of segments)
        seg_probs.append(i/total_num_segs)
    
    
    ## Let's write the actual loop now:
    
    counts = np.zeros(len(abundances))    # array of size 10 to hold our counts
    probs = np.zeros(N,dtype=object)     # array of size N to hold 10 probs for each read

    # do this for all reads:
    i = 0
    for read in R:
        joints = []  # temporary list to store 10 joints for each read
        # For each read, compute a log likelihood for each of the 10 arcs:
        j = 0 # counter variable that will range 0-9 to keep track of which arc we are dealing with
        for arc in arcs:
            if read in arc:
                numerator = (1/int(lengths[j]))*abundances[j]   # probability calculation
                denominator =  seg_probs[j]* np.sum(abundances)  # numerator summed over all transcripts
                joints.append(numerator/denominator)
            else:
                joints.append(0)   # if no probability
            j=j+1

        probs[i] = joints
        i+=1
    # Sum counts over all reads for each of the 10 transcripts:
    for i in range(len(probs)):
        for j in range(len(probs[0])):
            counts[j] = counts[j] + probs[i][j]
    
    
    return counts

Next, for our third step we write a function for maximization, which takes in counts as an input and returns an updated u (nucleotide abundances). These will be the new parameters theta.

In [16]:
def maximization(C):
    '''
    Given C (read counts) and 
    Caluclate the updated nu (nucleotide abundances)
    '''
    # Normalizing all the counts by dividing
    for i in range(len(counts)):
        counts[i] = counts[i]/N
    
    # changing the name
    abundances = counts
    
    return abundances

Now we run our expectation maximization until convergence.
We will also calculate the log likelihood after every EM pass so that we keep track of how we are doing.


Important note: Unlike k-means where we ran with 20 different initial conditions, here we are only running with one set of initial conditions because Li and Dewey proved there is only one optimum

I am doing it in two different ways:
1. As in Sean's solution, running the EM a large number of times (defined as Sean by 1000)
2. Using the more elegant convergence tresholds


First way:


In [ ]:
# Number of times we want to run this EM:
n = 1000

for i in range(n):  # absolute value of our treshold which is a percent change in ll 
    counts = expectation(sim_reads, theta) # obtain our counts by using our expectation function
    abundances = maximization(counts)  # normalize counts into abundances by using our maximization function
    theta[0][1] = abundances  # updating our theta parameters with the latest abundances
    ll = log_likelihood(sim_reads, theta)  # also calculating our log likelihood for every EM run
    
# this EM takes a bit to run, but it does converge to the right values

Second way: 

(leaving it commented out)

In [ ]:
# Setting arbitrary initial ll's to make sure our loop runs:
#old_ll = 10000000
#ll = 0 

# Setting a convergence treshold as a percentage change in ll
#treshold = 10

#while abs(ll - old_ll)  > treshold:  # absolute value of our treshold which is a percent change in ll 
    #counts = expectation(sim_reads, theta)
    #abundances = maximization(counts)
    #theta[0][1] = abundances  # updating our theta parameters with the latest abundances
    #ll = log_likelihood(sim_reads, theta)
    
    #return abundances,ll

Since we used nucleotide abundances above, we want to convert our results back to transcript abundances for our final step. This will help us look at the total numbers to analyze:

In [ ]:
# Convert nucleotide abundance to transcript abundance
taus = []  # empty list to store our abundances in after converting
for i in abundances:
    taus.append(i*N)
    
# Take a look at our estimates for the true taus:
print (taus)
print(tau)

**What do you think of Lestrade et al.'s conclusion that all 10 mRNA transcripts are expressed at about the same level? What are the most abundant two transcripts, and how much of the population to they account for? What are the least abundant two transcripts?**

This conclusion of Lestrade is a result of a poor implementation of EM. The 10 mRNA transcripts are definitely not expressed at about the same level 

Per our transcript abundances shown above, the two most abundand ones are H and G. The two least abundant ones are E and J.